In [3]:
import geopandas as gpd;
import geopy.distance;
import pandas as pd;
import numpy as np;

import warnings;
warnings.filterwarnings("ignore");

In [9]:
# Acquiring bounds using the properties in Ames in the dataset.
# lon_min = housing_coords[["longitude"]].min();
# lon_max = housing_coords[["longitude"]].max();
# lat_min = housing_coords[["latitude"]].min();
# lat_max = housing_coords[["latitude"]].max();

# Using Oren's boungs for Aimes
lon_min = -93.7638;
lon_max = -93.4449;
lat_min = 41.9503;
lat_max = 42.1089;


In [7]:
housing_coords = pd.read_csv('../data/house_coordinates.csv', index_col = 0);
housing_coords.drop_duplicates(inplace = True);
housing_coords.shape

(2579, 5)

In [ ]:
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)

# print(geopy.distance.geodesic(coords_1, coords_2).km)

In [2]:
# A collection of points of interests shape files

# Points of interest (2XXX): Subdivision to 2XYY is suggested
gis_osm_pois = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pois_a_free_1.shp');
gis_osm_pois2 = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pois_free_1.shp');

# Places of worship (3XXX)
# Known conflicts: Heartland Baptist Church (very likely duplicates)
gis_osm_pofw = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pofw_a_free_1.shp');
gis_osm_pofw2 = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pofw_free_1.shp');

# Natural (4XXX)
gis_osm_natural = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_natural_a_free_1.shp');
gis_osm_natural2 = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_natural_free_1.shp');

# Traffic (50XX-53XX)
gis_osm_traffic = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_traffic_a_free_1.shp');
gis_osm_traffic2 = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_traffic_free_1.shp');

# Transport infrastructure (56XX)
gis_osm_transport = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_transport_a_free_1.shp');
gis_osm_transport2 = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_transport_free_1.shp');

# Land use and land cover (7XXX)
gis_osm_landuse = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_landuse_a_free_1.shp');

# Bodies of water (8XXX)
gis_osm_water = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_water_a_free_1.shp');

#poi_crs = gis_osm_poi.to_crs(epsg=3857);

gpd_list = [gis_osm_pois, gis_osm_pois2, gis_osm_pofw, gis_osm_pofw2, gis_osm_natural, gis_osm_natural2,\
           gis_osm_traffic, gis_osm_traffic2, gis_osm_transport, gis_osm_transport2, gis_osm_landuse, gis_osm_water];

# Preprocess: add longitude and latitud, sieve range
# gis_osm_pofw2['longitude'] = gis_osm_pofw2['geometry'].x;
# gis_osm_pofw2['latitude'] = gis_osm_pofw2['geometry'].y;

# gis_osm_pofw2 = sieve_Ames_range(gis_osm_pofw2);

In [13]:
frame = pd.DataFrame({"osm_id": pd.Series([], dtype='int'), "code":pd.Series([], dtype='int'),\
                      "fclass": pd.Series([], dtype='str'), "name":pd.Series([], dtype='str'), "geometry":[],\
                      "longitude": pd.Series([], dtype='float'), "latitude": pd.Series([], dtype='float')});
# frame = gpd.GeoDataFrame(frame, crs="EPSG:4326");
for gpd_thing in gpd_list:
    df = prepare_gpd_vincinity(gpd_thing);
    frame = pd.concat([frame, df], axis = 0);

In [14]:
code_fclass_dict = {};
code_list = sorted(list(frame["code"].unique()));
for c in code_list:
    code_fclass_dict[c] = frame[frame["code"] == c][["fclass"]].iloc[0,0];


In [19]:

#temp = [w for w in code_list if (w >= 2000 and w <= 2200)];
temp = [w for w in code_list];
test_housing = housing_coords;

for w in temp:
    print("Current code is ", w);
    ref_frame = frame[frame["code"]==w];
    print("Mapping with ", ref_frame.shape[0], "GIS datapoints")
    %time test_housing = group_vincinity(test_housing, ref_frame, colname=str(w) + "_" + code_fclass_dict[w], mileage = 0.25);
    
test_housing.to_csv('../data/house_coordinates_0.25.csv', index = False)

Current code is  2001
Mapping with  2 GIS datapoints
Wall time: 2.89 s
Current code is  2002
Mapping with  6 GIS datapoints
Wall time: 5.1 s
Current code is  2004
Mapping with  2 GIS datapoints
Wall time: 3.31 s
Current code is  2005
Mapping with  7 GIS datapoints
Wall time: 5.76 s
Current code is  2007
Mapping with  4 GIS datapoints
Wall time: 4.22 s
Current code is  2008
Mapping with  3 GIS datapoints
Wall time: 3.54 s
Current code is  2013
Mapping with  2 GIS datapoints
Wall time: 2.91 s
Current code is  2015
Mapping with  10 GIS datapoints
Wall time: 7.24 s
Current code is  2081
Mapping with  2 GIS datapoints
Wall time: 2.98 s
Current code is  2082
Mapping with  22 GIS datapoints
Wall time: 14 s
Current code is  2101
Mapping with  7 GIS datapoints
Wall time: 6.01 s
Current code is  2110
Mapping with  2 GIS datapoints
Wall time: 3.01 s
Current code is  2120
Mapping with  4 GIS datapoints
Wall time: 4.16 s
Current code is  2121
Mapping with  4 GIS datapoints
Wall time: 4.25 s
Current

Wall time: 37.7 s
Current code is  7203
Mapping with  295 GIS datapoints
Wall time: 3min 2s
Current code is  7204
Mapping with  46 GIS datapoints
Wall time: 30.5 s
Current code is  7206
Mapping with  10 GIS datapoints
Wall time: 8.46 s
Current code is  7208
Mapping with  2 GIS datapoints
Wall time: 3.37 s
Current code is  7209
Mapping with  69 GIS datapoints
Wall time: 44.7 s
Current code is  7210
Mapping with  16 GIS datapoints
Wall time: 12.1 s
Current code is  7211
Mapping with  7 GIS datapoints
Wall time: 6.46 s
Current code is  7212
Mapping with  4 GIS datapoints
Wall time: 4.77 s
Current code is  7215
Mapping with  1 GIS datapoints
Wall time: 2.76 s
Current code is  7216
Mapping with  2 GIS datapoints
Wall time: 3.35 s
Current code is  7217
Mapping with  19 GIS datapoints
Wall time: 13.7 s
Current code is  7218
Mapping with  1602 GIS datapoints
Wall time: 16min 27s
Current code is  7228
Mapping with  176 GIS datapoints
Wall time: 1min 50s
Current code is  7229
Mapping with  306 G

In [16]:
test_housing.head()

,PID,Address,Coords4,latitude,longitude,2001_police,2002_fire_station,2004_post_box,2005_post_office,2007_library,2008_town_hall,2013_nursing_home,2015_graveyard,2081_university,2082_school,2101_pharmacy,2110_hospital,2120_doctors,2121_dentist,2129_veterinary
0,909176150,"436 HAYWARD AVE, Ames, Iowa, USA","436 Hayward Ave, Ames, IA 50014, USA",42.017780,-93.651452,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,911128020,"320 S 2ND ST, Ames, Iowa, USA","320 S 2nd St, Ames, IA 50010, USA",42.021389,-93.614855,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,535377150,"1524 DOUGLAS AVE, Ames, Iowa, USA","1524 Douglas Ave, Ames, IA 50010, USA",42.038070,-93.612065,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,534177230,"2304 FILLMORE AVE, Ames, Iowa, USA","2304 Fillmore Ave, Ames, IA 50010, USA",42.044900,-93.631893,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [12]:
print(lon_min, lon_max, lat_min, lat_max);

longitude   -93.693158
dtype: float64 longitude   -93.57949
dtype: float64 latitude    41.986321
dtype: float64 latitude    42.063401
dtype: float64


In [36]:
# Oren's legacy
# coords_1 = (gis_osm_poi["latitude"][0], gis_osm_poi["longitude"][0]);
# coords_2 = (gis_osm_poi["latitude"][1], gis_osm_poi["longitude"][1]);

# print(geopy.distance.geodesic(coords_1, coords_2).miles > 4);

# from geopy.distance import geodesic

False


In [12]:
def group_vincinity(df_house, df_place, colname="POI", mileage = 0.25):
    '''
    Input variables:
    df_house: Dataframe of the houses. Need to contain the columns ["longitude", "latitude"].
    df_place: Dataframe of places of interests. Need to contain the columns ["longitude", "latitude"].
    colname: A string, indicates the column name which will be attached to df_house.
    mileage: A float. Will look for all pairs of df_house and df_place within th mileage miles.
    
    Output variables:
    An updated dataframe of df_house which has a new column "POI".
    The column consists integers which indicates the number of 'df_place' within 'mileage' in each 'df_house'.
    '''
    df_house["gps"] = df_house[["latitude", "longitude"]].apply(lambda x: (x["latitude"], x["longitude"]), axis = 1);
    df_house[colname] = df_house[["gps"]].apply(lambda x: np.sum(df_place[["latitude", "longitude"]].\
    apply(lambda y: geopy.distance.geodesic((y["latitude"],y["longitude"]), x).miles < mileage, axis = 1)), axis = 1);
    df_house.drop("gps", axis = 1, inplace = True);
    return df_house;

In [10]:
def sieve_Ames_range(df_place):
    '''
    Input variables:
    df_place: A dataframe of places. Need to contain the columns ["longitude", "latitude"].
    
    Output variables:
    An updated dataframe. Rows out of the range will be deleted.
    The boundary latitudes and longitudes are given by the global variables "lon, lat"_"min, max".
    '''
    return df_place[(df_place["longitude"] >= lon_min) & (df_place["longitude"] <= lon_max)\
                   & (df_place["latitude"] >= lat_min) & (df_place["latitude"] <= lat_max)];

In [11]:
def prepare_gpd_vincinity(gpd_thread):
    '''
    Input variables:
    gpd_thread: a gpd object to be processed
    
    Output variable:
    A processed dataframe, with longitude and latitude columns ready, region within interest, and geometry object dropped.
    '''
    gpd_thread['longitude'] = gpd_thread['geometry'].centroid.x;
    gpd_thread['latitude'] = gpd_thread['geometry'].centroid.y;

    gpd_thread = sieve_Ames_range(gpd_thread);
    # gpd_thread.drop("geometry", axis = 1, inplace = True);
    return gpd_thread;

In [166]:
housing_coords.shape

(2579, 5)

In [95]:
%time group_vincinity(housing_coords, gis_osm_poi, "POI", 2.5);
# housing_coords.drop("gps", axis = 1, inplace = True)

Wall time: 16.8 s


,PID,Address,Coords4,latitude,longitude,POI
0,909176150,"436 HAYWARD AVE, Ames, Iowa, USA","436 Hayward Ave, Ames, IA 50014, USA",42.017780,-93.651452,7
1,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,6
2,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,6
3,911128020,"320 S 2ND ST, Ames, Iowa, USA","320 S 2nd St, Ames, IA 50010, USA",42.021389,-93.614855,5
4,535377150,"1524 DOUGLAS AVE, Ames, Iowa, USA","1524 Douglas Ave, Ames, IA 50010, USA",42.038070,-93.612065,5
...,...,...,...,...,...,...
2619,903205040,"1021 RIDGEWOOD AVE, Ames, Iowa, USA","1021 Ridgewood Ave, Ames, IA 50010, USA",42.031937,-93.626510,6
2620,905402060,"3619 MARY CIR, Ames, Iowa, USA","3619 Mary Cir, Ames, IA 50014, USA",42.027798,-93.666899,6
2621,909275030,"2140 SUNSET DR 2142, Ames, Iowa, USA","2140 Sunset Dr, Ames, IA 50014, USA",42.019944,-93.643206,8
2622,907192040,"5319 CLEMENS BLVD, Ames, Iowa, USA","5319 Clemens Blvd, Ames, IA 50014, USA",42.016826,-93.690382,3
